# Pretraining for ASR

In [2]:
# installing libs
# !pip3 install torch torchvision torchaudio datasets transformers soundfile jiwer --index-url https://download.pytorch.org/whl/cu118
# !pip3 install librosa --index-url https://pypi.org/simple
!pip3 install datasets evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import re
import torch
import torch.nn as nn
import numpy as np

from datasets import load_dataset, disable_caching
from evaluate import load
from transformers import Wav2Vec2ForPreTraining, Wav2Vec2FeatureExtractor, Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import Wav2Vec2Encoder


## Finetuning Wav2Vec2 model on CTC loss (5 points)


In this task you have to create pipeline for finetuning pretrained multilingual Wav2Vec2 model on belarusian audio from [Fleurs](https://huggingface.co/datasets/google/fleurs) dataset.

#### Prepare data

In [4]:
fleurs = load_dataset("google/fleurs", "be_by", split=["train", "validation", "test"], trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

fleurs.py:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

train.tar.gz:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

dev.tar.gz:   0%|          | 0.00/282M [00:00<?, ?B/s]

...

dev.tsv:   0%|          | 0.00/387k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/966k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
fleurs[0]["transcription"][9]

'вышыня двух пілонаў складае 83 метры даўжыня моста - 378 метраў праезная частка складаецца з дзвюх палос шырыня кожнай - 3,50 м'

In [6]:
fleurs[0][0]

{'id': 396,
 'num_samples': 250560,
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/116a91f1ff7dbe8f3be7feef0c5ec35adeeb1f92683504d263d4a1e3f588fd03/10009414287632395082.wav',
 'audio': {'path': 'train/10009414287632395082.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00031281,
         -0.00038069, -0.00132966]),
  'sampling_rate': 16000},
 'transcription': 'у той жа час паблізу ад верагодных маршрутаў уварвання базіравалася вельмі мала караблёў каралеўскага флоту таму што адміралы асцерагаліся іх патаплення нямецкімі паветранымі сіламі',
 'raw_transcription': 'У той жа час паблізу ад верагодных маршрутаў уварвання базіравалася вельмі мала караблёў каралеўскага флоту, таму што адміралы асцерагаліся іх патаплення нямецкімі паветранымі сіламі.',
 'gender': 1,
 'lang_id': 6,
 'language': 'Belarusian',
 'lang_group_id': 1}

In this task, you should:

* filter all samples, where `transcription` includes digits. Hint: take care of specific belarussian symbols "і", "ў";
* remove punctuation from `transcription`.

In [7]:
import re

has_digit = re.compile(r"\d")

def filter_f(x):
    print(x)
    return x is not None


preprocessed_train = fleurs[0].filter(lambda x: has_digit.search(x['transcription']) is None)
preprocessed_val = fleurs[1].filter(lambda x: has_digit.search(x['transcription']) is None)

Filter:   0%|          | 0/2433 [00:00<?, ? examples/s]

Filter:   0%|          | 0/408 [00:00<?, ? examples/s]

In [8]:
len(fleurs[0]), len(preprocessed_train), len(fleurs[1]), len(preprocessed_val)

(2433, 1927, 408, 355)

#### Train tokenizer

There you should train your own BPE tokenizer based on texts from Fleurs dataset using [HuggingFace tokenizer](https://huggingface.co/docs/tokenizers/en/training_from_memory).

In [9]:
from tokenizers import models, trainers, tokenizers, normalizers, pre_tokenizers, decoders

PAD_TOKEN = "[PAD]"
BOS_TOKEN = "[BOS]"
EOS_TOKEN = "[EOS]"
UNK_TOKEN = "[UNK]"
VOCAB_SIZE = 1000

tokenizer = tokenizers.Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.NFKC()
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()
trainer = trainers.BpeTrainer(special_tokens=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN, UNK_TOKEN], vocab_size=VOCAB_SIZE, show_progress=True)
tokenizer.train_from_iterator(preprocessed_train['transcription'], trainer)


In [10]:
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[{"id":0, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":1, "content":"[BOS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":2, "content":"[EOS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":3, "content":"[UNK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=NFKC(), pre_tokenizer=ByteLevel(add_prefix_space=True, trim_offsets=True, use_regex=True), post_processor=None, decoder=ByteLevel(add_prefix_space=True, trim_offsets=True, use_regex=True), model=BPE(dropout=None, unk_token=None, continuing_subword_prefix=None, end_of_word_suffix=None, fuse_unk=False, byte_fallback=False, ignore_merges=False, vocab={"[PAD]":0, "[BOS]":1, "[EOS]":2, "[UNK]":3, "!":4, "'":5, ",":6, "-":7, ".":8, "/":9, ":":10, "

#### Loading model and preprocessor

In [11]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
   "facebook/wav2vec2-xls-r-300m"
)
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    ctc_loss_reduction="mean",
    pad_token_id=tokenizer.token_to_id(PAD_TOKEN),
    vocab_size=tokenizer.get_vocab_size(),
)


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Data processor and data collator

In [12]:
class CtcDataProcessor:
    def __init__(self, tokenizer, feature_extractor):
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor

    def __call__(self, row):
        """
            Function applies tokenizer on row['transcription'] and applies feature extractor on audio column in row.
            Input: dict with transcription and audio fields
            Output: original dict includes `labels` column with tokenized sequence and `input_values` column with computed spectrogram.
        """
        row['labels'] = torch.tensor(self.tokenizer.encode(row['transcription'], add_special_tokens=True).ids)
        row['input_values'] = torch.tensor(self.feature_extractor(row['audio']['array'], sampling_rate=row['audio']['sampling_rate']).input_values[0])
        return row

In [13]:
data_processor = CtcDataProcessor(tokenizer, feature_extractor)
data_processor(preprocessed_train[0])

{'id': 396,
 'num_samples': 250560,
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/116a91f1ff7dbe8f3be7feef0c5ec35adeeb1f92683504d263d4a1e3f588fd03/10009414287632395082.wav',
 'audio': {'path': 'train/10009414287632395082.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00031281,
         -0.00038069, -0.00132966]),
  'sampling_rate': 16000},
 'transcription': 'у той жа час паблізу ад верагодных маршрутаў уварвання базіравалася вельмі мала караблёў каралеўскага флоту таму што адміралы асцерагаліся іх патаплення нямецкімі паветранымі сіламі',
 'raw_transcription': 'У той жа час паблізу ад верагодных маршрутаў уварвання базіравалася вельмі мала караблёў каралеўскага флоту, таму што адміралы асцерагаліся іх патаплення нямецкімі паветранымі сіламі.',
 'gender': 1,
 'lang_id': 6,
 'language': 'Belarusian',
 'lang_group_id': 1,
 'labels': tensor([141, 631, 232,  78, 304,  95, 805, 834, 153, 581, 132, 155, 210, 748,
         139, 229, 460, 141, 585, 100,

In [14]:
data_processor = CtcDataProcessor(tokenizer, feature_extractor)
train = preprocessed_train.map(data_processor, keep_in_memory=True, remove_columns=preprocessed_train.column_names)
val = preprocessed_val.map(data_processor, keep_in_memory=True, remove_columns=preprocessed_val.column_names)

Map:   0%|          | 0/1927 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

In [15]:
class CTCDataCollator:
    # HuggingFace requires pad transcript tokens with this value
    LABELS_PAD_IDX = -100

    @staticmethod
    def collate_tokens(tokens_batch, type, pad_value=0.0):
        """
            Function collates list of tokens
        """
        pass

    def __call__(self, batch):
        """
            Function collates `input_values` and `labels` into one tensor respectively
            Input: list with dicts, output of CTCDataProcessor
            Output row includes `labels` column with tokenized sequence, `input_values` column with computed spectrogram and
            `attention_mask` (0 for not-attending position, 1 for attending)
        """
        input_values = torch.nn.utils.rnn.pad_sequence([torch.tensor(row['input_values']) for row in batch], batch_first=True, padding_value=0.0)
        labels = torch.nn.utils.rnn.pad_sequence([torch.tensor(row['labels']) for row in batch], batch_first=True, padding_value=self.LABELS_PAD_IDX)
        attention_mask = (input_values != 0).float()
        return {"input_values": input_values, "labels": labels, "attention_mask": attention_mask}

In [16]:
batch = [train[0], train[1], train[2]]
collator = CTCDataCollator()
collated_batch = collator(batch)
collated_batch

{'input_values': tensor([[0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
         [0.0003, 0.0003, 0.0003,  ..., 0.0000, 0.0000, 0.0000],
         [0.0002, 0.0002, 0.0002,  ..., 0.0031, 0.0041, 0.0092]]),
 'labels': tensor([[ 141,  631,  232,   78,  304,   95,  805,  834,  153,  581,  132,  155,
           210,  748,  139,  229,  460,  141,  585,  100,  302,  127,  156,   85,
           300,  835,  572,  111,  338,  340,  135,   93,  173,  123,  340,  243,
           123,  782,  221,  298,  246,  419,  176,  153,  208,   88,  110,   83,
           297,   91,  149,  132,  642,  318,  371,  163,  667,  532,  283,   91,
           136,  208,  255,  775,  319,  537,  399,   93,  220],
         [ 264,  676,  365,  993,  658,  157,  140,   92,  165,  292,   92,   88,
           175,  407,  624,  207,  321,  126,  600,  601,  111,  175,   94,  159,
           100,   94,  730,  361,  750,  408,  125,  145,  118,  785,  229,  423,
           273,  785,   88,  117,  646,  105,   98, -100

#### Inference and metrics computing

There you should use simple greedy straregy for CTC output decoding.

Hint: Don't forget about padding value -100 in reference.

Hint: Don't forget about CTC output format.

In [17]:
wer_metric = load("wer")

class MetricsComputer:
    def __call__(self, pred):
        """
            Input: object with fields `predictions` for CTC model output and `label_ids` for tokenized reference;
            Output: dict with key `wer` and computed wer
        """
        # model prediction tensor, tensor batch_size x max_seq_len x vocab_size
        preds_logits = torch.tensor(pred.predictions)
        # reference, tensor batch_size x max_seq_len
        label_ids = torch.tensor(pred.label_ids)

        pad_token_id = tokenizer.token_to_id(PAD_TOKEN)
        preds = torch.argmax(preds_logits, dim=-1)
        label_str = [tokenizer.decode(ids[torch.where(ids != -100)].tolist()) for ids in label_ids]
        pred_str = []

        for seq in preds.tolist():
            pred = []
            previous = -100
            for id in seq:
                if id == previous:
                    continue
                if id != pad_token_id:
                    pred.append(id)
                previous = id
            pred_str.append(tokenizer.decode(pred))

        print(f"Prediction: {pred_str[0]}")
        print(f"Reference: {label_str[0]}")

        wer = wer_metric.compute(predictions=pred_str, references=label_str)
        return {"wer": wer}

In [18]:
predictions = torch.randn(1, 100, VOCAB_SIZE)
label_ids = torch.randint(0, VOCAB_SIZE, (1, 30))
class preds:
    predictions = predictions
    label_ids = label_ids

pred = preds()
metrics_computer = MetricsComputer()
metrics_computer(pred)

Prediction: ства зая ён пав асоб вцыі наз ўз напбы мал� знаходзіццауд вадры заянальскіх наведвзе які ўвстаўова зямон суецца мал цягам ёсць аг: хар план які ісарм пакло гэтым адпавед гівыч ан�ыхактыаюццазу тымвяр забн згадодшастаў� або толькібы цяж імскай месят ак сябеlнаванымч дааўніцтды вельмі міцьцесяарыствайартантэтер�ацелемктуагалеспсам
Reference:  магчым ам апецца у адн праві быцьгіняўася склад маюць адзіннікуавысць былі тамуце��стаўавед акрамя каліага дак пап пар


<ipython-input-17-7c4d3e19bc02>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_logits = torch.tensor(pred.predictions)
<ipython-input-17-7c4d3e19bc02>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_ids = torch.tensor(pred.label_ids)


{'wer': 2.588235294117647}

#### Overfitting on train batch

In this task you should check pipeline correctness by overfitting on you need to finetune Wav2Vec2 model and achieve 50 WER or lower accuracy on val set.

In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test",
    per_device_train_batch_size=1, # you could increase batch size
    gradient_accumulation_steps=8,
    eval_strategy="steps",
    max_steps=5000,
    fp16=True,
    save_steps=500,
    eval_steps=200,
    logging_steps=200,
    learning_rate=3e-4,
    weight_decay=1e-5,
    warmup_steps=200,
    gradient_checkpointing=True,
)

In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=CTCDataCollator(),
    args=training_args,
    compute_metrics=MetricsComputer(),
    train_dataset=train,
    eval_dataset=val,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: andreiyaremenko9 (andreiyaremenko9-yandex) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Wer
200,33.709500,6.344954,1.000000
400,7.773900,6.354747,1.000000
600,6.364800,6.329714,1.000000
800,6.362300,6.320075,1.000000
1000,6.353200,6.322417,1.000000
1200,6.352400,6.317117,1.000000


Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і 

Step,Training Loss,Validation Loss,Wer
200,33.709500,6.344954,1.000000
400,7.773900,6.354747,1.000000
600,6.364800,6.329714,1.000000
800,6.362300,6.320075,1.000000
1000,6.353200,6.322417,1.000000
1200,6.352400,6.317117,1.000000
1400,6.339800,6.306157,1.000000
1600,6.341300,6.315879,1.000000
1800,6.331300,6.301317,1.000000
2000,6.340900,6.308911,1.000000


Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
Prediction: 
Reference:  члены адной і той жа субкультуры часта вызначаюць сваю прыналежнасць да яе праз адметны і сімвалічны стыль у адзенні манеры паводзін і жаргоне
